# ENGRI 1120 Discussion 1: Fun With Total and Species Mass Balances

## Introduction
Suppose we interested in tracking the mass of each chemical component in a mixture of $\mathcal{M}$ components for a chemical reaction process e.g., the starting material, the final product and perhaps some by-products flowing into and from a chemical reactor unit. Then, we can write the _species mass balance_ for component $i$ as:

$$\sum_{s\in\mathcal{S}}\nu_{s}\dot{m}_{s,i} + \dot{m}_{G,i} = \frac{dm_{i}}{dt}\qquad{i=1,2,\dots,\mathcal{M}}$$

where $\mathcal{S}$ denotes the set of streams associated with the chemical reactor unit, $\dot{m}_{s,i}$ denotes the mass flow rate of component $i$ in stream $s$ (units: kg per hour), $\dot{m}_{G,i}$ denotes the generation rate for species $i$ (units: kg per hour) and $\nu_{s}$ denotes the stream direction paramater.

## Materials and Methods

In [7]:
# Setup My Julia Environment - add packages
import Pkg
Pkg.add("TOML")
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/Manifest.toml`


In [5]:
# import packages that we need -
using JLD2
using TOML
using Distributions

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1662


In [8]:
function simulate_species_mass_balances(config::Dict{String,Any}; σ::Float64 = 0.0) 
    
    # setup:
    # we have a mixing process
    # two streams in (A, B), one stream out (A+B)

    # setup the simulation parameters -
    T₀ = config["simulation"]["T1"]              # initial time
    T₁ = config["simulation"]["T2"]              # final time (units: hr)
    h = config["simulation"]["h"]                # timestep units: hr (equiv to 1 min)
    T = range(T₀, stop=T₁, step=h) |> collect    # simulation time range
    Z = Normal(0.0, 1.0)

    # how many states?
    ℳ = config["parameters"]["M"]

    # how many steps?
    ℒ = length(T)
    X = zeros(ℒ, ℳ)

    # setup the input and outputs -
    ṁ₁ = config["parameters"]["m_dot_in_1"]     # total mass flow rate stream 1 (units: kg/hr)
    ṁ₂ = config["parameters"]["m_dot_in_2"]     # total mass flow rate stream 2 (units: kg/hr)

    # mass fractions for the input streams 
    w_in_1 = config["parameters"]["w_in_1"]     # stream 1
    w_in_2 = config["parameters"]["w_in_2"]     # stream 2

    # κ value -
    κ = config["parameters"]["kappa"]

    # main loop -
    for t ∈ 1:(ℒ-1)
        for state = 1:ℳ

            # compute ṁ₃ -
            ṁ₃_i = κ * X[t, state]

            # update the state array -
            X[t+1, state] = X[t, state] + h * (ṁ₁ * w_in_1[state] + ṁ₂ * w_in_2[state] - ṁ₃_i) + σ * sqrt(h) * rand(Z)
        end
    end

    # return -
    return (T, X)
end

simulate_species_mass_balances (generic function with 1 method)

## Problems

### A) How long does it take the system to reach steady state?

### B) At steady state, is conservation of mass true?

### C) The client wants a different mixing ratio, how can we do that?